In [1]:
# from Pianki.Analiza_pianek import saldo, naliczone, wstrzymane
# from Modele_db import engine

# with engine.begin() as conn:
#     saldo.rename(columns={"KOD": "kod", "SALDO": "stan"}).to_sql("SALDO", conn, if_exists="replace")
#     naliczone.rename(columns={'LIMIT_NAZWA':"limit_nazwa", 'KOD':"kod", 'ZAPOT_ZLEC':"zapot_zlec", 'LIMIT_DATA_PROD':"limit_data_prod"}).to_sql("NALICZONE", conn, if_exists="replace")
#     wstrzymane.rename(columns={"KOD":"kod", 'ILOSC':"ilosc"}).to_sql("WSTRZYMANE", conn, if_exists="replace")

In [1]:
from Modele_db.modele_db import *
from Pianki.Analiza_pianek.funkcje_analizy_pianek import *

# pd.set_option()

In [2]:
Ogolna_analiza_objetosci()
print("---\nZAPEŁNIENIE MAGAZYNU")
Ogolna_analiza_objetosci("podsum_prc")

MAX_obj:                    3705
ZAMOWIONE_obj:              1070
CZEKA_NA_SPAKOWANIE_obj:    331
SALDO_obj:                  2060
WOLNE_obj:                  1730
WOLNE_NIE_SPAK_obj:         2061
DO_ZAM_obj:                 843
---
ZAPEŁNIENIE MAGAZYNU
SALDO / MAX: 55.6%
WOLNE / MAX: 46.7%
WOLNE_NIE_SPAK / MAX: 55.6%
ZAPEŁNIENIE MAG PO ZDJECIU 3 PACZEK RAZEM Z ZAMOWIENIAMI: 84.5%


In [3]:
# Podsumowanie_paczek_i_Pw()

In [5]:
br = Braki(WOLNE="SALDO")[0] #USUNAC PROBLEM Z DUBLUJACYMI SIE ZAMOWIANIAMI(JEDNO DOSTARCZONE DRUGIE W DRODZE TWORZĄ SIĘ DWIE POZYCJE)
br.sort_values(by=["GRUPA", "OPIS"])
# br.to_excel("BRAKI.xlsx")

In [6]:
Zagrozone(WOLNE="SALDO")
zgr = Zagrozone(False,WOLNE="SALDO")
# zgr
# zgr[((zgr.ZAMOWIONE + zgr.CZEKA_NA_SPAKOWANIE) == 0)]
# zgr[(zgr.SALDO < zgr.MIN)]
zgr[((zgr.ZAMOWIONE + zgr.CZEKA_NA_SPAKOWANIE) == 0)&(zgr.SALDO < zgr.MIN)]

WOLNE PONIZEJ MIN: 132 POZYCJE
WOLNE PONIZEJ MIN NIE ZAMOWIONE: 36 POZYCJE
SALDO PONIZEJ MIN: 109 POZYCJE
SALDO PONIZEJ MIN NIE ZAMOWIONE: 20 POZYCJE


,OPIS,ZAMOWIONE,CZEKA_NA_SPAKOWANIE,SALDO,MIN,WOLNE_SALDO
45,CALYPSO 1,0.0,0.0,9.0,18,9.0
53,CALYPSO [L2H,0.0,0.0,6.0,8,6.0
57,CALYPSO ][,0.0,0.0,27.0,30,26.0
59,DIVA X1X,0.0,0.0,2.0,3,1.0
68,DIVA 4X,0.0,0.0,3.0,4,2.0
73,DIVA P113x95,0.0,0.0,1.0,2,1.0
74,DIVA P95x95,0.0,0.0,7.0,8,6.0
85,"ELIXIR 2,5",0.0,0.0,21.0,22,16.0
94,"EXTREME 1,5s",0.0,0.0,0.0,8,0.0
97,"EXTREME [1,5s",0.0,0.0,0.0,8,0.0


In [6]:
#IMPORT ANAMLIZY PIANEK
from Pianki.Analiza_pianek.instrukcje_zamawiana import instrukcja_zamawiania_pianpol as izp, instrukcja_zamawiania_ciech as izc
from Pianki.Analiza_pianek.Podsumowanie_analizy_pianek import Podsumowanie_analizy_pianek

arp = Podsumowanie_analizy_pianek(izp)
# arc = Podsumowanie_analizy_pianek(izc)

In [ ]:
ava = arp["AVANT"].ar
ava.DO_ZAM_SZT.mean()

In [ ]:
ava[ava.DO_ZAM_SZT > ava.MIN]

In [ ]:
(arp > 1).query("GRUPA < 4")

In [ ]:
arp.Tabela_podsumowania_analizy#.query("GRUPA == 4")

In [ ]:
arp["STONE"].ar

In [ ]:
"sqlite:///Z:/450. PLANISTA - ZAOPATRZENIE/ARCHIWUM_ANALIZ.db"



In [ ]:
arp._Tabela_podsumowania_analizy.columns

In [4]:
Zapis_danych_do_Archiwum(243m, analiza, arp._Tabela_podsumowania_analizy, "sqlite:///Z:/450. PLANISTA - ZAOPATRZENIE/ARCHIWUM_ANALIZ.db")